**Содержание:**

1. Математическая постановка задачи

2. Решение задачи

3. Регуляризация - борьба с переобучением

4. Какое представление данных будет наилучшим, как их преобразовать

5. Реализация модели на numpy

6. Изучение готовых реализаций из библиотек

7. Обучение разных реализаций и сравнение

8. Как контролировать обучение модели

9. Как посмотреть важность признаков. Что ещё можно полезного извлечь из обученной модели

## **1. Математическая постановка задачи**

**Задача регрессии:** на вход подаётся вектор *$x ∈ ℝ^n$*. На выходе пораждается скалярное значение *$y ∈ ℝ$*. Линейная регрессия решает такую задачу
Обозначим *$\hat{y}$* – значение *$y$*, предсказанное моделью. Определим результат модели в виде *$\hat{y} = w^T*x$*, где *$w ∈ ℝ^n$* – вектор параметров. *$w$* - набор весов описывающих влияние отдельных признаков на результат предсказания.

Таким образом задача формулируется так: *Предсказать $y$ по $x$, вычислив значение $\hat{y} = w^T*x$*.

Также сформулировать задачу можно с помощью оценки максимального правдоподобия, но здесь такой подход подробно рассматриваться не будет

![Alt text](image.png)

## **2. Решение задачи**

Для подбора таких *$w^T$*, чтобы *$\hat{y}$* было как можно ближе к истинным значениям *$y$*, нужно ввести меру качества для *$w^T$*

Пусть имеется матрица плана с *$m$* примерами, которые мы будем использовать не для обучения, а только для оценки качества работы модели. Имеется также вектор меток, содержащий правильные значения *$y$* для каждого из этих примеров. Поскольку  этот набор данных будет использоваться только для контроля качества, назовем его тестовым набором. Обозначим матрицу плана *$X_{test}$*, а вектор меток регрессии – *$y_{test}$*. 

Один из способов измерения качества модели – вычислить среднеквадратическую ошибку модели на тестовом наборе. Если вектор *$y_{test}$* содержит предсказания модели на тестовом наборе, то среднеквадратическая ошибка определяется по формуле *$MSE_{test} = \frac{1}{m} \sum_{i=1}^{m}(\hat{y_{i}} - y_{i})^2$*

Эта мера ошибки обращается в *$0$*, когда *$\hat{y}_{test}$* = *$y_{test}$*. 

Кроме того, *$MSE_{test} = \frac{1}{m} \mid\mid \hat{y}_{test} - y_{test} \mid\mid^2_{2}$*, поэтому ошибка тем больше, чем больше евклидово расстояние между предсказаниями и метками

Мы должны спроектировать алгоритм машинного обучения, который улучшает веса *$w$* таким образом, что *$MSE_{test}$* уменьшается по мере того, как алгоритм получает новый опыт, наблюдая обучающий набор *$(X_{train}, y_{train})$*. Интуитивно понятный способ добиться этой цели – минимизировать среднеквадратическую ошибку на обучающем наборе, *$MSE_{train}$*. Для минимизации *$MSE_{train}$* нужно просто приравнять градиент к *$0$* и решить получившееся уравнение:

*$\triangledown_{w} MSE_{test} = 0$* *$\rArr$* *$\triangledown_{w} \frac{1}{m} \mid\mid \hat{y}_{test} - y_{test} \mid\mid^2_{2} = 0$* *$\rArr$* *$\triangledown_{w} (X_{train}*w - y_{train})^T(X_{train}*w - y_{train}) = 0$* *$\rArr$* *$\triangledown_{w}(w^T*X^T*X*w - 2*w^T*X^T*y + y^T*y) = 0$* 

*$\rArr$*  $\nabla\mid\mid X*w - y \mid\mid^2_{2} = 2*X^T*X*w - 2*X^T*y = 0$

$X^T*X*w = X^Ty$

$w = y*X^T(X^T*X)^{-1}$




Однако это решение считается неоптимальным, кроме того, для других моделей аналитического решения может и не существовать. К счастью, найти такой вектор *w*, при котором функция потерь минимальна, хоть и приближенно, позволяет градиентный спуск и его модификации.

*   обращение матрицы имеет сложность O($n^3$)
*   матрица $X^T*X$ может оказаться вырожденной, и тогда обращение будет невозможным
*   если заменить функционал MSE на другой, то решение для $w$, вероятнее всего, не получится

### **Градиентный спуск**

*$w^{k} = w^{k-1} - lr*\triangledown_{w_{k-1}}(L)$* Где lr - learning rate - длина шага, являкется гиперпараметром, задаётя разработчиком, может быть функцией.

![Alt text](image-10.png)

Простой градиентный спуск имеет множество недостатков, поэтому были придуманы его модификации, которые будут рассмотрены в реализации технологий глубокого обучения, а пока опишу их кратко.

![Alt text](image-14.png)

![Alt text](image-15.png)

### **Оценка решения задачи линейной регрессии - метрики и функции потерь**

**Функция потерь:** Функция потерь определяет, насколько хорошо модель справляется с текущей задачей обучения. Ее цель - создать градиент, который модель будет использовать для обновления своих параметров в процессе обучения. Функция потерь должна быть дифференцируемой и ее минимизация приводит к улучшению качества модели.

**Метрика:** Метрика, с другой стороны, представляет собой способ оценки производительности модели после обучения. Метрика измеряет, насколько хорошо модель работает на тестовых данных или в реальных условиях. Метрика может быть не дифференцируемой и не использоваться в процессе оптимизации параметров модели.

MSE также является и метрикой, по которой оценивают результат работы линейной регрессии. Кроме метрики MSE есть множество других метрик, просто MSE наиболее популярная. И хотя MSE дифференцируемая, выпуклая и обусловленная вероятностным подходом к решению данной задачи, у неё есть ряд проблем. Рассмотрим другие оценки (функции потерь по совместительству) и сравним между собой

*MSE = $\frac{1}{n} \sum_{i=1}^{n}(\hat{y_{i}} - y_{i})^2 → min$* : плохо интерпретируема, так как возвращает возведённую в квадрат оценку искомой величины

*RMSE = $\sqrt{\frac{1}{n}\sum_{i=1}^{n}(\hat{y_{i}} - y_{i})^2} → min$* : RMSE решает проблему квадрата величины, но всё ещё эта метрика остаётся плохо интерпертируемой. Например RMSE = 500 будет считаться очень плохим решением задачи, если целевая переменная расположена в промежутке от 0 до 200. Но такое значение ошибки будет считатья великолепным, если целевая переменная расположена в промежутке от 10000 до 1000000

*$R^2 = 1 - \frac{\sum_{i=1}^{n}(\hat{y_{i}} - y_{i})^2}{\sum_{i=1}^{n}({y_{i}} - \bar{y_{i}})^2}$, где $\bar{y_{i}}$* - среднее значение целевой переменной.
Если *$\hat{y_{i}} = y_{i}$*, то *$R^2$* равен *1* - это означает, что модели идеальна.

Если модель выдаёт постоянное значение, равное среднему значению целевой переменной на выборке, то *$R^2$* будет равен *0*

Модель считается приемлемой, если её *$R^2$* лежит в пределах од 0 до 1. И чем ближе к единице тем лучше

Как видно, эта метрика хорошо интерпретируема. И всё-таки, для разных задач значение R^2 можно интерпертировать по-разному.

*MAE = $\frac{1}{n}\sum_{i=1}^{n}\mid \hat{y_{i}} - y_{i} \mid → min$* : Немного более устойчива к выбросам в данных. Сложна для градиентной оптимизации.

![Alt text](image-5.png) - сравнение MAE и MSE в выборке с выбросом для двух моделей. MSE уменьшает большую ошибку - ошибку на выбросе. MAE не приоретезирует ошибки на выбросах.

*HUBER LOSS* - ![Alt text](image-6.png) - компромисс между MSE и MAE. Не имеет второй производной. Дельта определяет, какие данные считаются выбросами, а какие - нет.
 
![Alt text](image-7.png) - график Huber loss

*Log-cosh = $log(cosh(a-y))$* - имеет вторую производную, похожа на HUBER LOSS.

![Alt text](image-8.png)

*MSLE = $(log(a+1) - log(y+1))^2$* - работает, если прогнозы модели и целевая переменная неотрицательны.  Она может быть полезной, когда важно учитывать относительные изменения величин, а не абсолютные значения.

*MAPE = $\mid\mid \frac{y-a}{a}\mid\mid$* - MAPE измеряет ошибку в процентах относительно истинных значений целевой переменной. Она показывает относительную точность прогнозов. Минус - несимметрична.

![Alt text](image-9.png)

Пример: Прогноз продаж товаров: Предположим, вы управляете магазином и хотите прогнозировать ежедневные продажи различных товаров. В этой задаче MAPE может помочь вам измерить точность в процентах, насколько ваши прогнозы близки к фактическим продажам.



*SMAPE*, или Симметричное среднее абсолютное процентное отклонение (Symmetric Mean Absolute Percentage Error), это метрика оценки точности прогнозов в задачах прогнозирования временных рядов и прогнозирования, особенно в сферах, где важна относительная оценка ошибки в процентах. SMAPE измеряет процентное отклонение между истинными значениями и прогнозами модели.

![Alt text](image-13.png)

*Квантильная функция потерь (Quantile Loss)* - позволяет регулировать штраф за занижение и повышение прогноза. max(q * (y-y_pred), (1-q) * (y_pred-y))

Функция потерь подбирается под особенности задачи.

## **3. Регуляризация - борьба с переобучением**

В зависимости от факторов таких как размер датасета, , модель может показывать очень хорошие результаты на тренировочной выборке, но на тестовой показывать неудовлетворительные результаты. Такакя ситуация называется переобучением. При переобучении модель просто подстраивается под данные обучающей выборки, а не находит закономерности в данных. 

![Alt text](image-16.png)

Для борьбы с этой проблемой используется регуляризация. Один из способов - это штрафовать модель за слишком большие веса. Чаще всего для таких целей используют L1 - (Lasso) и L2 - (Ridge)- регуляризации.

![Alt text](image-17.png)
![Alt text](image-18.png)

Лямбда - коэф-т регуляризации, гиперпараметр, задаваемый вручную, подбором.

Также существует кобинация L1 и L2, называемая Elastic-Net регуляризацией

$\mid\mid X*w - y \mid\mid^2_{2} + λ*\mid\mid w\mid\mid_1^2 → min$ - Lasso-регрессия

$\mid\mid X*w - y \mid\mid^2_{2} + λ*\mid\mid w\mid\mid_2^2 → min$ - Ridge-регрессия

$\mid\mid X*w - y \mid\mid^2_{2} + λ*\mid\mid w\mid\mid_2^2 + λ*\mid\mid w\mid\mid_1^2→ min$ - Elastic Net-регрессия

![](image-19.png) - изменение весов разными видами регуляризации. Как видно, L1 хороша при отборе признаков, когда из большого кол-ва нужно выбрать самые важные, ведь она зануляет веса.

**1! На смещение, т.е w0 регуляризация действовать не должна! 2! Чтобы регуляризация работала, признаки при входе в модель должны быть отнормированы!**

![Alt text](image-3.png)

![Alt text](image-1.png) - математическое обоснование L1 и L2 регуляризаций

![Alt text](image-2.png)

## **4. Какое представление данных будет наилучшим, как их преобразовать**

Как мы уже поняли, чтобы модель не переобучилось нужно применять регуляризацию, а для регуляризации нужно подавать отнормированные признаки. Т.е ![Alt text](image-21.png)

Кроме того, в данных могут быть пропущенные значения, и в зависимотси от данных нужно их заполнить или вовсе удалить.
Иногда бывает полезно составлять новые признаки из существующих, а старые удалять, если они коррелируют. (Feature-Engineering)

Предобработка данных - очень масштабная тема, поэтому здесь будут изложены краткие принципы.

**Предобработка данных:**

*   Заполнение пропусков (чаще всего средним значением или медианой)
*   Кодировка (OneHotEncoder, LabelEncoder, BinaryEncoder, HelmetEncoder, Backward-Difference Encoder, TargetEncoding ...)
*   Масштабирование признаков (StandardDcaler, MinMaxScaler)
*   Добавление признаков (полиномиальные признаки, взятие логарифма, квадратного корня, применение тригонометрических функий)
*   Удаление выбросов/шума

## **5. Реализация модели**

In [2]:
import numpy as np

In [6]:
class Custom_LinearRegression:
    def __init__(self, lr = 0.01, rate_forgetting = 0.05, iters = 1000, reg_param = 0.1, reg_type = None, loss_fn_type = 'MSE'):
        self.lr = lr
        self.iters = iters
        self.reg_param = reg_param
        self.reg_type = reg_type
        self.loss_fn_type = loss_fn_type
        self.rate_forgetting = rate_forgetting
        self.weights = None
        self.bias = 0
        self.losses = []
    
    def __loss_fn(self, y, preds):
        if self.reg_type == None:
            return np.sum((y-preds)**2)
        elif self.reg_type in ['l1', 'lasso']:
            return np.sum((y - preds)**2) + self.reg_param*np.sum(np.abs(self.weights))
        elif self.reg_type in ['l2', 'ridge']:
            return np.sum((y - preds)**2) + self.reg_param*np.sum((self.weights)**2)
        elif self.reg_type == 'elastic':
            return np.sum((y - preds)**2) + self.reg_param*np.sum((self.weights)**2) + (1-self.reg_param)*np.sum(np.abs(self.weights))
        else:
            raise ValueError("Введите 'l1', 'l2', 'lasso', 'ridge', или 'elastic'")

    def fit(self, X, y):
        num_samples, num_features = X.shape
        self.weights = np.zeros(num_features)
        functional_evaluation = (1/num_samples)*__loss_fn(y, X.T*self.weights)
        for i in range(self.iters):
            pass

In [5]:
weights = np.array([0,1,3])
y = 0.4
print(np.sum(np.abs(weights))*y)

1.6


![Alt text](image-4.png)